## Package imports

In [2]:
# Backbone imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
# Single Cell imports
import anndata
import scanpy as sc
import scrublet as scr
import harmonypy
from pybiomart import Server

import warnings
warnings.filterwarnings(action="ignore")

## Package Settings

In [2]:
sc.settings.verbosity = 2
sc.logging.print_header()

scanpy==1.8.2 anndata==0.8.0 umap==0.5.3 numpy==1.22.4 scipy==1.8.1 pandas==1.4.2 scikit-learn==1.0.2 statsmodels==0.13.2 python-igraph==0.9.10 pynndescent==0.5.7


In [3]:

data_dir = Path("C:/Users/gperry/OneDrive - The Jackson Laboratory/Documents/GSBE Project - meeting recordings/molecule_files")


## Functions

In [4]:
#### Query Functions ####

def query_ribosomal_genes(biomart_annos):
    """feeds biomart_annos into function, """
    return biomart_annos.loc[             # returns the location in biomart
        biomart_annos.go_id.isin(["GO:0005840"]) |   #This GO ID must be 005840
        biomart_annos.external_gene_name.str.lower().str.match("^m?rp[ls]\d+"), # \d+ one or more digits
        "external_gene_name"
    ].dropna().unique()
def query_mitochondrial_genes(biomart_annos):
    return biomart_annos.loc[
        biomart_annos.chromosome_name.str.match("^(mt|MT)") & True, 
        "external_gene_name"
    ].dropna().unique()
def query_hemoglobin_genes(biomart_annos):
    return biomart_annos.loc[
        biomart_annos.go_id.isin(["GO:0005833"]),
        "external_gene_name"
    ].dropna().unique()
def query_proliferation_genes(biomart_annos):
    return biomart_annos.loc[
        biomart_annos.go_id.isin(["GO:0008283"]),
        "external_gene_name"
    ].dropna().unique()
def query_xlinked_genes(biomart_annos):
    return biomart_annos.loc[
        biomart_annos.chromosome_name.str.match("^(chrX|X|x|chrx)") & True, 
        "external_gene_name"
    ].dropna().unique()
def query_ylinked_genes(biomart_annos):
    return biomart_annos.loc[
        biomart_annos.chromosome_name.str.match("^(chrY|Y|y|chry)") & True, 
        "external_gene_name"
    ].dropna().unique()
#### Annotation Functions #####################################################################
def annotate_var(adata, species):
    biomart_annos = sc.queries.biomart_annotations(
        species, 
        ["external_gene_name", "chromosome_name", "go_id"], 
        use_cache=True
    )
    #cc_genes = pd.Index(pd.read_csv("cell_cycle.csv", header=0, index_col=1).index[:484])
    #cc_genes = cc_genes.tolist() + cc_genes.str.capitalize().tolist()
    #stress_genes = pd.read_csv("coregene_stress-response.csv", header=0, index_col=6).index
    #stress_genes = stress_genes.tolist() + stress_genes.str.capitalize().tolist()
    idx = adata.var.species.isin([species])
    adata.var.loc[idx, "hemoglobin"] = adata.var_names.isin(query_hemoglobin_genes(biomart_annos))[idx]
    adata.var.loc[idx, "mitochondrial"] = adata.var_names.isin(query_mitochondrial_genes(biomart_annos))[idx]
    adata.var.loc[idx, "ribosomal"] = adata.var_names.isin(query_ribosomal_genes(biomart_annos))[idx]
    adata.var.loc[idx, "x_linked"] = adata.var_names.isin(query_xlinked_genes(biomart_annos))[idx]
    adata.var.loc[idx, "y_linked"] = adata.var_names.isin(query_ylinked_genes(biomart_annos))[idx]
    adata.var.loc[idx, "sex_linked"] = adata.var_names.isin(["Xist", "XIST"])[idx] | adata.var.y_linked[idx]
    adata.var.loc[idx, "proliferation"] = adata.var_names.isin(query_proliferation_genes(biomart_annos))[idx]
    #adata.var.loc[idx, "cell_cycle"] = adata.var_names.isin(cc_genes)[idx]
    #adata.var.loc[idx, "stress_response"] = adata.var_names.isin(stress_genes)[idx]
    adata.var.loc[idx, "exclude_from_highly_variable"] =  \
        adata.var.proliferation[idx] | \
        adata.var.hemoglobin[idx] | \
        adata.var.mitochondrial[idx] | \
        adata.var.ribosomal[idx] | \
        adata.var.sex_linked[idx]
        #adata.var.stress_response[idx] | \
        #adata.var.cell_cycle[idx] | \       
################################################################################   
def annotate_raw_10x_data(output_path: Path, species: str):
    """
    Use to load and quickly annotate a filtered_feature_bc_matrix.h5
    """
    assert species in ["mmusculus", "hsapiens", "hsapiens+mmusculus"]
    output_path = Path(output_path)
    filtered_h5_files = list(output_path.rglob("filtered_*.h5"))
    if filtered_h5_files:
        filtered_h5_file = filtered_h5_files[0]
    else:
        raise FileNotFoundError(f"Could not find 'filtered_*.h5' under {output_path}")
    output_id = output_path.stem
    adata = sc.read_10x_h5(filtered_h5_file)
    soupX_dirs = list(output_path.rglob("soupX"))
    if soupX_dirs:
        soupX_dir = soupX_dirs[0]
        adata.layers["soupX"] = sc.read_10x_mtx(soupX_dir).X.copy()
    else:
        adata.layers["soupX"] = adata.X.copy()
    adata.obs["processed_id"] = output_id
    seqsat_files = list(output_path.rglob("sequencing_saturation.csv"))
    if seqsat_files:
        seqsat = pd.read_csv(seqsat_files[0], index_col=0, header=0)
        adata.obs.loc[seqsat.index, "sequencing_saturation"] = seqsat["saturation"].values 
    for key in ["hemoglobin", "mitochondrial", "ribosomal", "x_linked", "y_linked", "sex_linked", "exclude_from_highly_variable"]:
        adata.var[key] = False    
    if species == "hsapiens+mmusculus":
        adata.var["is_human"] = adata.var.genome.isin(["GRCh38"])
        adata.var["species"] = "mmusculus"
        adata.var.loc[adata.var.is_human, "species"] = "hsapiens"
    
        adata.var_names = adata.var_names.str.extract("(GRCh38|mm10)_+(.*)").iloc[:,1]
    else:
        adata.var["is_human"] = species == "hsapiens"
        adata.var["species"] = species
    adata.var_names_make_unique()  
    for specie in species.split("+"):
        annotate_var(adata, specie)             # this calls annotate_var() then runs qc metrics
    sc.pp.calculate_qc_metrics(
        adata,
        qc_vars=("mitochondrial", "hemoglobin", "ribosomal", "sex_linked", "is_human"),
        percent_top=None,
        log1p=True,
        inplace=True
    )  
    scrub = scr.Scrublet(adata.X)
    doublet_scores, predicted_doublets = scrub.scrub_doublets()
    adata.obs["scrublet_putative_doublet"] = predicted_doublets
    adata.obs["scrublet_score"] = doublet_scores  
    adata.raw = adata
    adata.layers["raw"] = adata.X.copy()
    adata.layers["log_raw"] = sc.pp.log1p(adata.X,copy=True).copy()
    adata.layers["log_soupX"] = sc.pp.log1p(adata.layers["soupX"],copy=True)  
    return adata
#### QC Functions ###############################################################################

def qc_metrics_violin(adata):
    titles = ["UMIs", "Genes","% mtRNA", "% rRNA", "Hemoglobin"]
    keys = [
        "total_counts", "n_genes_by_counts", "pct_counts_mitochondrial", 
        "pct_counts_ribosomal", "total_counts_hemoglobin"
    ]
    L = len(keys)
    fig, axs = plt.subplots(1, L, figsize=(L*2, 3), dpi=200, gridspec_kw=dict(wspace=1))

    for ax, key, title in zip(axs.flat, keys, titles):
        sc.pl.violin(adata, key, layer="log_raw", color="0.85", use_raw=False, ax=ax, show=False, size=0.5, jitter=0.4, stripplot=False)
        ax.set_xlabel(""); ax.set_ylabel(adata.obs.processed_id.head(1).values[0])
        ax.set_xticks([])
        ax.set_title(title)
        sns.despine(fig, ax)
    fig.subplots_adjust(wspace=1)
    return fig

## Import Directory and sample import

In [5]:
# output id as well as short hand mapping we might use later
output_id_mapping = {
    "PR19050": "primary_tumor_1", 
    "PR19051": "primary_tumor_2", 
    "PR19052": "primary_tumor_3", 
    "SC2100407": "pdx_nsg-il6_1", 
    "SC2100408": "pdx_nsg", 
    "SC2100424": "pdx_nsg-il6_2",
    "SC2100426": "pdo"
}

In [6]:
input_paths = [data_dir / output_id for output_id in output_id_mapping.keys()]  #

In [7]:
species = "hsapiens+mmusculus"
raws = [annotate_raw_10x_data(p, species) for p in input_paths]    # This is running annotate_raw_10x_data() and setting it equal to "raws"

reading C:\Users\gperry\OneDrive - The Jackson Laboratory\Documents\GSBE Project - meeting recordings\molecule_files\PR19050\filtered_feature_bc_matrix.h5
 (0:00:01)
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.41
Detected doublet rate = 1.1%
Estimated detectable doublet fraction = 25.1%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 4.5%
Elapsed time: 13.2 seconds
reading C:\Users\gperry\OneDrive - The Jackson Laboratory\Documents\GSBE Project - meeting recordings\molecule_files\PR19051\filtered_feature_bc_matrix.h5
 (0:00:00)
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.41
Detected doublet rate = 1.9%
Estimated detectable doublet fraction = 34.7%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 5.6%
Elapsed time: 17.9 seconds
reading C:\Users\gperr

# Merging datasets
#### Simple concatenation along the genes. No Batch correction yet.

In [8]:
# Don't remove any genes here (as it will make integration harder)
# But remove cells based on whatever you want- filtering out basic cells not of interest
qc_params = {
    "min_genes_per_cell": 800,
    "max_pct_counts_mitochondrial": 20,
    "max_counts_hemoglobin": 50, 
}

qcs = []
for raw in raws:
    qc = raw.copy()
    sc.pp.filter_cells(qc, min_genes=qc_params.get("min_genes_per_cell", 800))
    idx = qc.obs.pct_counts_mitochondrial < qc_params.get("max_pct_counts_mitochondrial", 30)
    idx &= qc.obs.total_counts_hemoglobin < qc_params.get("max_counts_hemoglobin", 100)
    qc = qc[idx, :].copy()
    qcs.append(qc)

filtered out 648 cells that have less than 800 genes expressed
filtered out 1121 cells that have less than 800 genes expressed
filtered out 1730 cells that have less than 800 genes expressed
filtered out 1035 cells that have less than 800 genes expressed
filtered out 826 cells that have less than 800 genes expressed
filtered out 635 cells that have less than 800 genes expressed
filtered out 787 cells that have less than 800 genes expressed


In [9]:
merged_raw = anndata.concat(
    qcs, join="outer", label="dataset", keys=output_id_mapping.values(), index_unique="-", 
)
merged_raw.raw = merged_raw.copy()
merged_raw.layers["raw"] = merged_raw.X.copy()

In [10]:
output_path = data_dir / "merged_raw.h5ad"

In [11]:
merged_raw.write(output_path)